In [34]:
## Import the required libraries

import duckdb
import pandas as pd
from plotnine import (
    ggplot,
    aes,
    geom_bar,
    theme,
    element_text
)

df = pd.read_csv('./data/cleandata.csv', encoding='latin-1')

In [ ]:
dependency = duckdb.sql('''
select Partner_Countries,
    sum(Value) /
        (select
            sum(Value)
            from df
            where Year = 2021
            and Element = 'Import Value'
            and Reporter_Countries = 'United Arab Emirates'
        ) as imp_prop
    from df
    where Year = 2021
    and Element = 'Import Value'
    and Reporter_Countries = 'United Arab Emirates'
    group by Partner_Countries
    having imp_prop > 0
    order by imp_prop desc;
    ''').fetchdf().head(21)

In [101]:

top_imports = duckdb.sql('''
select Reporter_Countries,
    sum(Value) as imports_uae
    from df
    where Year = 2021
    and Element = 'Import Value'
    and Partner_Countries = 'United Arab Emirates'
    group by Reporter_Countries
    having imports_uae > 0
    order by imports_uae desc;
    ''').fetchdf().head(21)

total_imports = duckdb.sql('''
select Reporter_Countries,
    sum(Value) as import_value
    from df
    where Year = 2021
    and Element = 'Import Value'
    group by Reporter_Countries
    having import_value > 0
    order by import_value desc;
    ''').fetchdf()

dependants = duckdb.sql(
    '''
    drop table if exists imps;
    drop table if exists dependants;

    create table imps as
        select * from top_imports
        left join total_imports
        on top_imports.Reporter_Countries = total_imports.Reporter_Countries
        order by import_value desc;

        select Reporter_Countries,
            imports_uae / import_value as dep_ratio
            from imps
            where dep_ratio >= 0.04
            order by dep_ratio desc;
    ''').fetchdf()

dependants

,Reporter_Countries,dep_ratio
0,Oman,0.350573
1,Maldives,0.203470
2,Afghanistan,0.186384
3,Bahrain,0.175980
4,Syrian Arab Republic,0.173320
5,Kuwait,0.155162
6,Libya,0.144555
7,Iran (Islamic Republic of),0.107553
8,Yemen,0.101324
9,Benin,0.091926
